# Claude Tools - Basic Colab Integration

Simple integration for single Claude instance with Google Colab

## Setup Instructions

1. **Run this notebook in Google Colab**
2. **Mount Google Drive when prompted**
3. **The processor will start automatically**


In [ ]:
# Install dependencies
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client python-dotenv

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted")

In [ ]:
# Claude Tools Processor
import os
import json
import time
import subprocess
import sys
from io import StringIO
from contextlib import redirect_stdout, redirect_stderr

class ClaudeToolsProcessor:
    def __init__(self):
        self.session_id = f"colab_{int(time.time())}"
        print(f"🚀 Claude Tools Processor: {self.session_id}")
        
    def process_command(self, command):
        """Process commands from Claude"""
        try:
            cmd_type = command.get('type')
            
            if cmd_type == 'execute_code':
                return self._execute_code(command['code'])
            elif cmd_type == 'install_package':
                return self._install_packages(command['packages'])
            else:
                return {'error': f'Unknown command: {cmd_type}'}
                
        except Exception as e:
            return {'error': str(e)}
    
    def _execute_code(self, code):
        """Execute Python code"""
        stdout_capture = StringIO()
        stderr_capture = StringIO()
        
        try:
            with redirect_stdout(stdout_capture), redirect_stderr(stderr_capture):
                exec(code, globals())
            
            return {
                'success': True,
                'output': stdout_capture.getvalue(),
                'error': stderr_capture.getvalue() or None
            }
            
        except Exception as e:
            return {
                'success': False,
                'output': stdout_capture.getvalue(),
                'error': str(e)
            }
    
    def _install_packages(self, packages):
        """Install packages"""
        if isinstance(packages, str):
            packages = [packages]
        
        try:
            for package in packages:
                !pip install {package}
            
            return {
                'success': True,
                'output': f'Installed: {", ".join(packages)}'
            }
        except Exception as e:
            return {'success': False, 'error': str(e)}

# Initialize processor
processor = ClaudeToolsProcessor()

In [ ]:
# Start command monitoring
commands_folder = "/content/drive/MyDrive/claude-tools-commands"
os.makedirs(commands_folder, exist_ok=True)

print(f"👁️ Monitoring: {commands_folder}")
print("📱 Claude instances can now send commands")
print("🛑 Press Ctrl+C to stop")

processed = set()

try:
    while True:
        # Check for new commands
        for filename in os.listdir(commands_folder):
            if filename.startswith('command_') and filename.endswith('.json'):
                filepath = os.path.join(commands_folder, filename)
                
                if filepath in processed:
                    continue
                
                # Process command
                with open(filepath, 'r') as f:
                    command = json.load(f)
                
                print(f"📝 Processing: {command.get('type')}")
                result = processor.process_command(command)
                
                # Save result
                result_file = filename.replace('command_', 'result_')
                result_path = os.path.join(commands_folder, result_file)
                
                with open(result_path, 'w') as f:
                    json.dump(result, f)
                
                # Clean up
                os.remove(filepath)
                processed.add(filepath)
                
                print(f"✅ Completed: {result.get('success', False)}")
        
        time.sleep(1)
        
except KeyboardInterrupt:
    print("\n🛑 Processor stopped")
except Exception as e:
    print(f"❌ Error: {e}")